In [3]:
from pymongo import MongoClient
from bson.objectid import ObjectId

import time,datetime
import os 
import schedule

import pandas as pd
import pandas_datareader.data as web
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style

from copy import deepcopy
from IPython.display import clear_output

In [6]:
conn = MongoClient('120.126.136.17')
db = conn.Tracker
collection = [db.User_Info]# db.db2, db.dn2, db.james, db.leo
userInfo = []
for col in collection:
    cursor = col.find({})
    df_user = pd.DataFrame(list(cursor))

for username in df_user['username']:
    if username == 'pitest' or username == 'jack' or  username == 'manager':
        continue
    collection = [db[username]]
#     collection = [db.james]# db.db2, db.dn2, db.james, db.leo
    clean_data = []
    for col in collection:
        cursor = col.find({})
        df = pd.DataFrame(list(cursor))

    df.replace('', np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)

    df['hr_value'] = df['hr_value'].astype(float)
    df['o2_value'] = df['o2_value'].astype(float)
    df['latitude'] = df['latitude'].astype(float)
    df['longitude'] = df['longitude'].astype(float)
    df['step_value'] = df['step_value'].astype(float)
    df = df[(df['hr_value'] != 0)]
    # map out the weekday
    df['weekday'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).strftime('%w'))
    df['week'] = df['timestamp'].apply(lambda x: datetime.datetime.fromtimestamp(x).isocalendar()[1]
              - (datetime.datetime.fromtimestamp(x).isoweekday() < 7)) # do this so that week may start on monday

    dfsteptotal = pd.DataFrame({})
    dfsteptotal = df[['week', 'weekday']].set_index(['week', 'weekday'])
    dfsteptotal = dfsteptotal[~dfsteptotal.index.duplicated(keep='first')]
    earlestweek = df['week'].values[0]
    latestweek = df['week'].values[-1]
    addindex = pd.DataFrame({})
    for i in range(earlestweek, latestweek):
        for j in range(7):
            if not (i,str(j)) in dfsteptotal.index:
                add = pd.DataFrame({'week':[i],'weekday':[str(j)]})
                addindex = addindex.append(add)
    todayweekday = int(datetime.datetime.now().strftime('%w'))
    for j in range(int(todayweekday)):
        if not (latestweek,str(j)) in dfsteptotal.index:
            add = pd.DataFrame({'week':[latestweek],'weekday':[str(j)]})
            addindex = addindex.append(add)
    dfsteptotal.reset_index(inplace=True)
    dfsteptotal = dfsteptotal.append(addindex)
    dfsteptotal = dfsteptotal.set_index(['week', 'weekday'])
    dfsteptotal = dfsteptotal.sort_index()
    dfsteptotal['steps'] = 0

    # here I map between integer and the sting it's mean
    timeslidemap = {0:'morning', 1:'afternoon', 2:'evening'}
    basestamp = df['timestamp'][0]
    endstamp = df['timestamp'][-1:].values[0]
    # for num, row in df[:30000].iterrows():
    while True:
        try:
            dfbetween = df.loc[df['timestamp'].between(
                datetime.datetime.fromtimestamp(basestamp).replace(hour=0,minute=0,second=0).timestamp(),
                datetime.datetime.fromtimestamp(basestamp).replace(hour=23,minute=59,second=59).timestamp()
                        , inclusive=True)]
            steps = dfbetween['step_value'].sum()
        #     print(int(datetime.datetime.fromtimestamp(basestamp).strftime('%m')),
        #             int(datetime.datetime.fromtimestamp(basestamp).strftime('%d')),
        #                 timeslidemap[estimedevide])

            dfsteptotal.loc[[(datetime.datetime.fromtimestamp(basestamp).isocalendar()[1] - 
                                        (datetime.datetime.fromtimestamp(basestamp).isoweekday() < 7),
                                datetime.datetime.fromtimestamp(basestamp).strftime('%w'))], 
                                               ['steps']] += steps

            basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
            if basestamp > endstamp:
                break
        except Exception as e:
            print(str(e))
            basestamp = (datetime.datetime.fromtimestamp(basestamp) + datetime.timedelta(days=1)).timestamp()
    #     else:
    #         print(datetime.datetime.fromtimestamp(basestamp) ,datetime.datetime.fromtimestamp(endstamp))|

In [5]:
dfsteptotal

steps
week weekday        
12   0           0.0
     1           0.0
     2           0.0
     3         295.0
     4           0.0
     5           0.0
     6           0.0
13   0           0.0
     1        1972.0
     2         500.0
     3           0.0
     4           0.0
     5           0.0
     6         606.0
14   0           0.0
     1           0.0
     2           0.0
     3           0.0
     4           0.0
     5           0.0
     6           0.0
15   0         821.0
     1        3452.0
     2        2031.0
     3           0.0
     4        3764.0
     5        3301.0
     6        2332.0
16   0           0.0
     1           0.0
     2           0.0
     3           0.0
     4           0.0
     5        3480.0
     6           0.0
17   0        3263.0
     1        3797.0
     2        1043.0
     3        2507.0
     4         144.0
     5        2772.0
     6        4760.0
18   0           0.0
     1        3909.0
     2        4443.0
     3        4477.0
     4        2673.0
     5           0.0
     6        5004.0
19   0           0.0
     1           0.0